# Imports

In [36]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.impute import SimpleImputer

# Data Prep


In [37]:
data = pd.read_csv('../TrainDataset2024.csv', index_col=0)

#MISSING DATA
data.replace(999, np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
data = pd.DataFrame(imputer.fit_transform(data), columns=data.columns,index=data.index)


target = data[['pCR (outcome)']]#'RelapseFreeSurvival (outcome)']]
data.drop(columns=['pCR (outcome)','RelapseFreeSurvival (outcome)'], axis=1, inplace=True)

key_features = data[['ER', 'HER2', 'Gene']]
data.drop(columns=['ER', 'HER2', 'Gene'], axis=1, inplace=True)


In [38]:

#NORMALISATION
normalizer = Normalizer()
vector_normalized_data = normalizer.fit_transform(data)


In [39]:

#FEATURE REDUCTION
pca = PCA(n_components=0.95)
data_reduced = pca.fit_transform(vector_normalized_data)

pca_complete = pd.DataFrame(data_reduced, index=data.index)
pca_complete = pd.concat([pca_complete, key_features], axis=1)


# Training Data

In [40]:
param_grid = {
    'C': [0.1, 1, 10], 
    'gamma': [0.001, 0.01, 0.1], 
    'kernel': ['rbf', 'linear'] 
}

#CHANGE ME
model = SVC(class_weight='balanced')


In [ ]:
n_folds = 4

def train_model(data):
    # Outer K-fold cross-validation
    outer_cv = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    data = data.rename(str,axis="columns") 
    
    # Perform nested cross-validation
    outer_results = []
    for train_idx, test_idx in outer_cv.split(data, target):
        # Split data
        X_train, X_test = data.iloc[train_idx], data.iloc[test_idx]
        y_train, y_test = np.ravel(target.iloc[train_idx]), np.ravel(target.iloc[test_idx])

        # Inner loop: Hyperparameter tuning using GridSearchCV
        inner_cv = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv)
        grid_search.fit(X_train, y_train)

        # Evaluate on the test set
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        # Collect results
        outer_results.append({
            "best_params": grid_search.best_params_,
            "classification_report": classification_report(y_test, y_pred, output_dict=True, zero_division=0),
            "accuracy": best_model.score(X_test, y_test)
        })

    # Print the results for each fold
    for i, result in enumerate(outer_results, 1):
        print(f"Fold {i}")
        print(f"Best Parameters: {result['best_params']}")
        print("Classification Report:")
        print(classification_report(
            y_test, y_pred, target_names=['0.0', '1.0'], zero_division=0
        ))
        print(f"Accuracy: {result['accuracy']:.4f}")
        print("-" * 40)

    # Overall results
    mean_accuracy = sum(res["accuracy"] for res in outer_results) / len(outer_results)
    print(f"Mean Accuracy: {mean_accuracy:.4f}")


In [42]:
train_model(pca_complete)

Fold 1
Best Parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86        80
         1.0       0.48      0.65      0.55        20

    accuracy                           0.79       100
   macro avg       0.69      0.74      0.71       100
weighted avg       0.82      0.79      0.80       100

Accuracy: 0.7600
----------------------------------------
Fold 2
Best Parameters: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'}
Classification Report:
              precision    recall  f1-score   support

         0.0       0.90      0.82      0.86        80
         1.0       0.48      0.65      0.55        20

    accuracy                           0.79       100
   macro avg       0.69      0.74      0.71       100
weighted avg       0.82      0.79      0.80       100

Accuracy: 0.2300
----------------------------------------
Fold 3
Best Parameters: {'C': 0.1, 'gamma': 0.1, 'k

# Re-Train Single Final Model on entire dataset using best-performing hyperparameters